In [13]:
import MetaTrader5 as mt5
import pandas as pd

In [2]:
mt5.initialize(path="D:/Dev/Metatrader 5 (VTB Forex)/terminal64.exe")

True

In [14]:
def __get_last_rates__(count):
    mt5rates = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_H1, 0, count)
    rates = pd.DataFrame(
        mt5rates,
        columns=[
            "time",
            "open",
            "high",
            "low",
            "close",
            "tickvol",
            "spread",
            "real_volume",
        ],
    )
    # logging.debug("Получено " + str(len(rates)) + " котировок")
    return rates

In [16]:
print(__get_last_rates__(16))

          time     open     high      low    close tickvol  spread  \
0   1623049200  1.21667  1.21698  1.21633  1.21671     NaN      20   
1   1623052800  1.21670  1.21671  1.21570  1.21571     NaN      20   
2   1623056400  1.21570  1.21613  1.21506  1.21517     NaN      20   
3   1623060000  1.21516  1.21670  1.21440  1.21669     NaN      20   
4   1623063600  1.21668  1.21668  1.21511  1.21611     NaN      20   
5   1623067200  1.21611  1.21634  1.21518  1.21599     NaN      20   
6   1623070800  1.21599  1.21619  1.21496  1.21550     NaN      20   
7   1623074400  1.21551  1.21751  1.21540  1.21639     NaN      20   
8   1623078000  1.21640  1.21676  1.21592  1.21625     NaN      20   
9   1623081600  1.21624  1.21806  1.21605  1.21803     NaN      20   
10  1623085200  1.21804  1.21908  1.21728  1.21867     NaN      20   
11  1623088800  1.21866  1.21993  1.21845  1.21976     NaN      20   
12  1623092400  1.21974  1.22009  1.21928  1.21938     NaN      20   
13  1623096000  1.21